In [1]:
import numpy as np

# General Imports:
import tensorflow as tf
import numpy as np
import copy
import math

# From Imports:
from tensorflow.keras import layers, activations, initializers, regularizers, constraints, Model

from skimage.filters import threshold_otsu

@tf.custom_gradient
def custom_round(x):
	output = tf.keras.backend.round(x)
	def grad(dy):
		return dy#*(tf.keras.backend.maximum(0*x, 1-2*tf.keras.backend.abs(x-0.5)) + tf.keras.backend.maximum(0*x, 1-2*tf.keras.backend.abs(x+0.5)))
	return output, grad

class CWTConv2D(layers.Layer) :
	
	def __init__(self, filters, kernel_size, strides, use_bias=True, **kwargs):
		super(CWTConv2D, self).__init__(**kwargs)
		
		self.filters = filters
		self.kernel_size = kernel_size
		self.strides = strides
		self.use_bias = use_bias

	def build(self, input_shape):
		
		self.kernel = self.add_weight(name='kernel',
										shape=(self.kernel_size[0], self.kernel_size[1], input_shape[-1], self.filters),
										initializer=initializers.RandomNormal(mean=0.0, stddev=0.5, seed=0),
										trainable=True)

		if (self.use_bias) :							   
			self.biases = self.add_weight(name='biases',
											shape=(self.filters),
											initializer='zeros',
											trainable=True)
										  
		super(CWTConv2D, self).build(input_shape)
										  
	def call(self, inputs):
		
		inputs = custom_round(inputs)
		
		kernel = self.kernel
		
		#kernel = tf.keras.backend.round(kernel)
		kernel = custom_round(kernel)
		
		kernel = tf.keras.backend.clip(kernel, -1, 1)
		
		output = tf.keras.backend.conv2d(inputs, kernel=kernel, strides=self.strides, padding='valid', data_format="channels_last")
		
		if (self.use_bias) :
			biases = self.biases
			
			#biases = tf.keras.backend.round(biases)
			biases = custom_round(biases)
			
			biases = tf.keras.backend.clip(biases, -128, 127)
			
			output = tf.keras.backend.bias_add(output, biases, data_format='channels_last')
		
		output = tf.keras.backend.in_train_phase(activations.sigmoid(output), tf.dtypes.cast(tf.math.greater_equal(output, 0.0), tf.float32))
		
		#output = tf.keras.backend.clip(output, 0, 1)
		
		#output = tf.keras.backend.round(output)
		#output = custom_round(output)
		
		return output
												 
	def compute_output_shape(self, input_shape):
		
		return ((inputs.shape[0] - self.kernel_size[0])/self.strides[0] + 1, (inputs.shape[1] - self.kernel_size[1])/self.strides[1] + 1, self.filters)
	
class SpikeData(layers.Layer) :
	
	def __init__(self, **kwargs):
		super(CWTConv2D, self).__init__(**kwargs)
		

	def build(self, input_shape):
										  
		super(CWTConv2D, self).build(input_shape)
										  
	def call(self, inputs):

		output = custom_round(inputs)
		
		output = tf.keras.backend.clip(output, 0, 1)
		
		return output
												 
	def compute_output_shape(self, input_shape):
		
		return input_shape.shape
	
class CWTMaxPooling2D(layers.Layer) :
	
	def __init__(self, pool_size, strides, **kwargs):
		super(CWTMaxPooling2D, self).__init__(**kwargs)
		
		self.pool_size = pool_size
		self.strides = strides
		# only supports valid padding

	def build(self, input_shape):
										  
		super(CWTMaxPooling2D, self).build(input_shape)
										  
	def call(self, inputs):
		
		output = custom_round(inputs)
		
		output = tf.keras.backend.pool2d(output, pool_size=self.pool_size, strides=self.strides, padding='valid', data_format="channels_last")
		
		return output
												 
	def compute_output_shape(self, input_shape):
		
		return ((inputs.shape[0] - self.pool_size[0])/self.strides[0] + 1, (inputs.shape[1] - self.pool_size[1])/self.strides[1] + 1, self.filters)
	
	
# custom cwt constrain function
def constrain_weights_cwt(model) :

	# Slam the convolutional kernel weights to -1, 0, or 1
	weights = model.get_weights()
	orig_weights = copy.deepcopy(weights)
	names = [weight.name for layer in model.layers for weight in layer.weights]
	
	layer_num = -1
	for weight, name in zip(weights, names):
		layer_num += 1

		if 'cwt' in name and 'kernel' in name:
			weight = tf.keras.backend.round(weight)
			weight = tf.keras.backend.clip(weight, -1, 1)
			weights[layer_num] = weight
			
			
		if 'cwt' in name and 'bias' in name:	
			weight = tf.keras.backend.round(weight)
			weight = tf.keras.backend.clip(weight, -128, 127)
			weights[layer_num] = weight
			
	return orig_weights, weights
	
# rounds a Keras Conv2D layer's weights to a ternary value (-1, 0, 1) to 
# be compatible with deployment onto a SNN based simulator/enviornment
# finds the average weight, and then uses the mean as a reference point for where it should go
# this should be deprecated once the train while constrain approach is implemented
def constrain_weights(model) :

	# Slam the convolutional kernel weights to -1, 0, or 1
	weights = model.get_weights()
	orig_weights = copy.deepcopy(weights)
	names = [weight.name for layer in model.layers for weight in layer.weights]
	
	layer_num = -1
	for weight, name in zip(weights, names):
		layer_num += 1

		if 'kernel' in name:	
			old_shape = weight.shape
			flattened_weights = weight.flatten()
			absolute_weights = np.absolute(flattened_weights)
			mean = np.mean(absolute_weights)
			for i in range(len(flattened_weights)) :
				if (flattened_weights[i] > mean) :
					flattened_weights[i] = 1
				elif (flattened_weights[i] < -1 * mean) :
					flattened_weights[i] = -1
				else :
					flattened_weights[i] = 0
			weight = flattened_weights.reshape(old_shape)
			weights[layer_num] = weight
			
			
		if 'bias' in name:	
			old_shape = weight.shape
			flattened_weights = weight.flatten()
			absolute_weights = np.absolute(flattened_weights)
			mean = np.mean(absolute_weights)
			for i in range(len(flattened_weights)) :
				if (flattened_weights[i] > mean) :
					flattened_weights[i] = 1
				elif (flattened_weights[i] < -1 * mean) :
					flattened_weights[i] = -1
				else :
					flattened_weights[i] = 0
			weight = flattened_weights.reshape(old_shape)
			weights[layer_num] = weight
			
	return orig_weights, weights	
	
def print_weights(model) :
	view_weights = model.get_weights()
	names = [weight.name for layer in model.layers for weight in layer.weights]
	for weight, name in zip(view_weights, names) :
		if 'kernel' in name:	
			print(name + " weights: ")
			print(weight.shape)
			print_weights = weight
			
	
	for f in range(print_weights.shape[3]) :
		for y in range(print_weights.shape[0]) :
			line = ""
			for x in range(print_weights.shape[1]) :
				for c in range (print_weights.shape[2]) :
					line += str(print_weights[y][x][c][f])
				line += " "
			print(line)
		print("Next Filter: ")
		
	for weight, name in zip(view_weights, names) :
		if 'bias' in name:	
			print(name + " weights : ")
			print(weight)
			
"""
takes as input a numpy array of 2d color images, and applies optimal 
color thresholding to them, transforming them to binary images
"""
def opt_thresh_color(x_in):

	x_bin = np.zeros_like(x_in[:,:,:,0])
	x = np.copy(x_in)
	for i in np.arange(len(x)):
		for color in np.arange(3):
			thresh = threshold_otsu(x[i,:,:,color])
			x[i,:,:,color] = x[i,:,:,color] > thresh
		x_bin[i] = np.logical_or(np.logical_or(np.logical_and(x[i,:,:,0],x[i,:,:,1]),
											   np.logical_and(x[i,:,:,1],x[i,:,:,2])),
								 np.logical_and(x[i,:,:,0],x[i,:,:,2]))

	return x_bin
	
"""
takes as input a numpy array of 2d color images, and applies optimal 
color thresholding to them, transforming them to binary images
"""
def opt_thresh_color_three(x_in):

	x_bin = np.zeros_like(x_in[:,:,:,0])
	x = np.copy(x_in)
	for i in np.arange(len(x)):
		for color in np.arange(3):
			thresh = threshold_otsu(x[i,:,:,color])
			x[i,:,:,color] = x[i,:,:,color] > thresh
	return x

In [2]:
# Contains the Code for Tea Layer for use in TeaLarning and TensorFlow 2.0


# Future Imports:
from __future__ import absolute_import, division, print_function

# General Imports:
import tensorflow as tf
import numpy as np

# From Imports:
from tensorflow.keras import layers, activations, initializers, regularizers, constraints
#from tensorflow.keras import backend as K
# Custom Round Gradient:
# In a TeaLayer the connections are rounded during the feedforward process.
# We need a custom rounding function to implement this.
# For this function the gradient is treated as a [ 1 ] so as not to effect backprop.

@tf.custom_gradient
def CustomRound(x):
	output = tf.keras.backend.round(x)
	def grad(dy):
		return dy
	return output, grad
######################################################################################################################################
class Tea(layers.Layer):
	"""
	The following is an implementation of a TeaLayer used to implement IBM's TeaLarning Training
		method for RANC-based TrueNorth deployment.
	For this to be compatable with the TrueNorth architecture, some irregular constraints and 
		functionalities must be implemented.
	Each layer contains stastically initialized [ weights ], which are multiplied by trainable
		[ connections ]. [ Connections ] are floating point values which represent the
		probablity that a [ connection ] exists. When feeding-forward [ connections ] are normally
		constrained to the values of 1, if >= 0.5, and 0, else. 
		This method allows for feed-forward to represent actual TrueNorth computations, but still
		allows connections to be trained during backprop.
	Additionally, inputs into the TrueNorth layer must be constrained to binary spikes of 1 or 0.
		Input data is normalized between 0 and 1, then if the value is 0.5 or greater it is represented
		as a spike (1), otherwise it is represented as a non-spike (0).
	Finally, outputs of the layer must be constrained during validation and testing. After weighted
		inputs are calculated, each value is set to 1 if it is greater than or equal to 0, and 0 otherwise.
		During training, this is estimated by a sigmoid activation function.
	"""

	"""
	New TeaLayer Initialization:
	Arguments:
		units -- The number of neurons to use for this layer.
	Keyword Arguments:
		activation -- The type of activation function to use to estimate spiking during training.
					  Note: Sigmoid activation function is specifically chosen to most accurately
						represent spiking. This value [should] be left as the default.
					  DEFAULT: [sigmoid]
		use_bias   -- To use biases or not.
					  DEFAULT: [True]
		weight_initializer -- The initializer to use when initializing the weights. If [None], then
							  the function [tea_weight_initializer] is used. This function sets all
							  weights to be -1 or 1 as outlined by IBM's TeaLarning Literature.
							  DEFAULT: [None]
		bias_initalizer -- The initializer to use when initializing biases.
						   DEFAULT: [zeros]
		connection_initializer -- The initializer to use when initializing connection values.
								  Note: Connections should be initialized as a [probability distribution].
								  By default they are sampled from a normal distrubtion with a mean of 0.5
								  DEFAULT: [None]
		connection_regularizer -- A regularizer to use on the connection values, if any.
								  DEFAULT: [None]
		
		connection_constraint -- A constraint to apply to the connections, if any.
								 DEFAULT: [None]
		round_input -- When feeding-forward this option dictates to round the input values or not to. This should
					   be set to [True] to maintain spiking simulation.
					   DEFAULT: [True]
		
		round_connections -- When feeding-forward this option dictates to round the connection values or not.
							 DEFAULT: [True]
		clip_connection -- This option dictates if connection values should be clipped between 0 and 1 when feeding-forward.
						   DEFAULT: [True]
		round_bias -- This option dictates if biases should be rounded when feeding-forward.
					  DEFAULT: [True]
		constrain_after_train -- This option dictates if outputs should be constrained to spikes (0 or 1) when training is completed.
								 DEFAULT: [True]
	"""
	##################################################################################################################################
	def __init__(self,
				 units,
				 activation='sigmoid',
				 use_bias=True,
				 weight_initializer=None,
				 bias_initializer='zeros',
				 connection_initializer=None,
				 connection_regularizer=None,
				 connection_constraint=None,
				 round_input=True,
				 round_connections=True,
				 clip_connections=True,
				 round_bias=True,
				 constrain_after_train=True,
				 **kwargs):
		super(Tea, self).__init__(**kwargs)
		
		self.units=units
		
		self.activation=activations.get(activation)
		
		self.use_bias=use_bias

		if connection_initializer:
			self.connection_initializer=connection_initializer
		else:
			self.connection_initializer=initializers.TruncatedNormal(mean=0.5, seed=0)

		if weight_initializer:
			self.weight_initializer=weight_initializer
		else:
			self.weight_initializer=tea_weight_initializer

		self.bias_initializer=bias_initializer

		self.connection_regularizer=connection_regularizer
		
		self.connection_constraint=connection_constraint

		self.input_width=None

		self.round_input=round_input

		self.round_connections=round_connections
		
		self.clip_connections=clip_connections

		self.round_bias=round_bias

		self.constrain_after_train=constrain_after_train

		self.uses_learning_phase=True
	##################################################################################################################################
	def build(self, input_shape):
		assert len(input_shape) >= 2

		super(Tea,self).build(input_shape)

		shape = (input_shape[-1], self.units)

		self.static_weights = self.add_weight(name='weights',
											  shape=shape,
											  initializer=self.weight_initializer,
											  trainable=False)

		self.connections = self.add_weight(name='connections',
										   shape=shape,
										   initializer=self.connection_initializer,
										   regularizer=self.connection_regularizer,
										   constraint=self.connection_constraint)

		if self.use_bias:
			self.biases = self.add_weight(name='bias',
										  shape=(self.units,),
										  initializer=self.bias_initializer)
	##################################################################################################################################
	def call(self, inputs):

		# Constrain the Input:
		if self.round_input:
			inputs = CustomRound(inputs)
		else:
			inputs = tf.keras.backend.in_train_phase(inputs, CustomRound(inputs))

		# Connection Constraints:
		connections = self.connections

		if self.round_connections:
			connections = CustomRound(connections)
		else:
			connections = tf.keras.backend.in_train_phase(connections, CustomRound(connections))

		if self.clip_connections:
			connections = tf.keras.backend.clip(connections, 0, 1)
		else:
			connections = tf.keras.backend.in_train_phase(connections, tf.keras.backend.clip(connections, 0, 1))

		# Multiply Connections with Weights:
		weighted_connections = connections * self.static_weights

		# Dot Product the Input with the Weighted Connections
		output = tf.keras.backend.dot(inputs, weighted_connections)

		# Add biases if they are being used:
		if self.use_bias:
			
			# Constrain the biases first:
			if self.round_bias:
				biases = CustomRound(self.biases)
			else:
				biases = tf.keras.backend.in_train_phase(self.biases, CustomRound(self.biases))

			output = tf.keras.backend.bias_add(output, biases, data_format='channels_last')

		# Apply activation / Spike(s)
		output = tf.keras.backend.in_train_phase(self.activation(output), 
												 tf.dtypes.cast(tf.math.greater_equal(output, 0.0), tf.float32))

		return output
	##################################################################################################################################
	def compute_output_shape(self, input_shape):
		assert input_shape and len(input_shape) >= 2
		assert input_shape[-1]

		output_shape = list(input_shape)

		output_shape[-1] = self.units

		return tuple(output_shape)
	##################################################################################################################################
# END CLASS : TEA
"""
This function returns a tensor of alternating 1s and -1s. This is a basic re-implementation of IBM's own weight matrix initializations.
Argument:
	shape -- The shape of the weights to be initialized.
Keyword Arguments:
	dtype -- The data type to be used when initializing the weights.
			 DEFAULT : [np.float32]
"""
def tea_weight_initializer(shape, dtype=np.float32):
	num_axons = shape[0]
	num_neurons = shape[1]
	if dtype == 'float32':
		dtype = np.float32
	ret_array = np.zeros((int(num_axons), int(num_neurons)), dtype=dtype)

	for axon_num, axon in enumerate(ret_array):
		if axon_num % 2 == 0:
			for neuron in range(len(axon)):
				ret_array[axon_num][neuron] = 1
		else:
			for neuron in range(len(axon)):
				ret_array[axon_num][neuron] = -1

	return tf.convert_to_tensor(ret_array)

In [3]:
# Contains the code for the additive pooling layer required by a Tea Layer when using TeaLearning.
#
#
#
#

# Future Calls:
from __future__ import absolute_import, division, print_function

# Imports:
import tensorflow as tf
import numpy as np

# From Imports:
from tensorflow.keras import layers

"""
Additive Pooling Class:
    A helper layer designed to format data for output during the TeaLarning process.

    If the input data to the layer has multiple spikers per classification, then for each
    tick the spikes must be summed up. Then, once all neurons that correspond to a certain class
    have finished spiking, their sums will dictate the results for each class.

    Neurons are assumed to be arracnged such that each [num_class] represents a guess for each
    of the classes.

    For example:
        If we have 10 classes, and we are using 250 neurons. Then we would have something like:
        neuron_number: 0    1   2   3   4   5   6   7   8   9   10  11  12  13  ...
        class:         0    1   2   3   4   5   6   7   8   9   0   1   2   3   ...
"""
######################################################################################################################################
class AdditivePooling(layers.Layer):
    """
    Initializer for new AdditivePooling Layer.

    Arguments:
        num_classes -- The number of classes to be output'd
    """
    ##################################################################################################################################
    def __init__(self,
                 num_classes,
                 use_additive_pooling_processing=False,
                 add_pool_process_max=128,
                 **kwargs):
        super(AdditivePooling, self).__init__(**kwargs)

        self.num_classes=num_classes
        
        self.num_inputs=None
        
        self.use_additive_pooling_processing=use_additive_pooling_processing
        
        self.add_pool_process_max=add_pool_process_max
    ##################################################################################################################################
    def build(self, input_shape):
        assert len(input_shape) >= 2

        # The number of neurons must be collapsable into the number of classes.
        # i.e if we have 10 classes, the number of neurons must be a divisor of 10.
        assert input_shape[-1] % self.num_classes == 0

        self.num_inputs = input_shape[-1]
    ##################################################################################################################################
    def call(self, inputs):
        if len(inputs.shape) >= 3:
            output = tf.keras.backend.sum(inputs, axis=1)
        else:
            output = inputs

        # Reshape the outputs:
        output = tf.reshape(output, [-1, int(self.num_inputs/self.num_classes), self.num_classes])

        # Sum up the neurons
        output = tf.math.reduce_sum(output, 1)

        if self.use_additive_pooling_processing:
            # Scale the ouputs between 0 and add_pool_process_max:
            max_val = tf.constant(self.add_pool_process_max, dtype=tf.float32)
            max_output = tf.stack([tf.reduce_max(output,1) for i in range(self.num_classes)], axis=1)
            max_output = tf.math.divide(max_val, max_output)
            output = tf.math.multiply(output, max_output)

            # Convert any NaN's to 0:
            output = tf.where(tf.math.is_nan(output), tf.zeros_like(output), output)

        return output
    ##################################################################################################################################
    def computer_output_shape(self, input_shape):
        output_shape = list(input_shape)

        # Last dimension will be number of classes:
        output_shape[-1] = self.num_classes

        # Ticks were summed, so delete tick dimension if they exist:
        if len(output_shape) >= 3:
            del output_shape[1]

        return tuple(output_shape)

In [4]:
def opt_thresh_color(x_in):
  """
  takes as input a numpy array of 2d color images, and applies optimal 
  color thresholding to them, transforming them to binary images
  """
  x_bin = np.zeros_like(x_in[:,:,:,0])
  x = np.copy(x_in)
  for i in np.arange(len(x)):
    for color in np.arange(3):
      thresh = threshold_otsu(x[i,:,:,color])
      x[i,:,:,color] = x[i,:,:,color] > thresh
    x_bin[i] = np.logical_or(np.logical_or(np.logical_and(x[i,:,:,0],x[i,:,:,1]),
                                           np.logical_and(x[i,:,:,1],x[i,:,:,2])),
                             np.logical_and(x[i,:,:,0],x[i,:,:,2]))

  return x_bin

In [5]:
def opt_thresh_color_three(x_in):
  """
  takes as input a numpy array of 2d color images, and applies optimal 
  color thresholding to them, transforming them to binary images
  """
  #x_bin = np.zeros_like(x_in[:,:,:,0])
  x = np.copy(x_in)
  for i in np.arange(len(x)):
    for color in np.arange(3):
      thresh = threshold_otsu(x[i,:,:,color])
      x[i,:,:,color] = x[i,:,:,color] > thresh
    #x_bin[i] = np.logical_or(np.logical_or(np.logical_and(x[i,:,:,0],x[i,:,:,1]),
    #                                       np.logical_and(x[i,:,:,1],x[i,:,:,2])),
    #                         np.logical_and(x[i,:,:,0],x[i,:,:,2]))

  return x

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import cifar10
from keras.models import Sequential
from keras import activations, losses
from keras.layers import Conv2D, Flatten, Input, Activation, Dense, Dropout, MaxPooling2D
from keras.utils import np_utils
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from skimage.filters import threshold_otsu
 



In [7]:

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = opt_thresh_color_three(x_train)
x_test = opt_thresh_color_three(x_test)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)



In [8]:
  
inputs = Input(shape=(32,32,3))

# network = Conv2D(10, (7,7), name='1cwt', activation=activations.relu, kernel_regularizer='L2')(inputs)

# maxpool = MaxPooling2D()(network)

# network2 = Conv2D(15, (5,5), name='cwt', activation=activations.relu)(maxpool)

# maxpool2 = MaxPooling2D()(network2)

network3 = Conv2D(15, (11,11), name='2cwt', activation=activations.relu)(inputs)

maxpool3 = MaxPooling2D()(network3)

droppy = Dropout(0.25)(maxpool3)

flattened = Flatten()(droppy)

dense = Tea(units=120, name="tea_2")(flattened)

pooling = AdditivePooling(10)(dense)

predictions = Activation('softmax')(pooling)

model = Model(inputs=inputs, outputs=predictions)

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

x_train = x_train.reshape(-1, 32, 32, 3) 
x_test = x_test.reshape(-1, 32, 32, 3)

model.fit(x_train, y_train, batch_size=128, epochs=100, verbose=1, validation_split=0.2)

score = model.evaluate(x_test, y_test, verbose=0)

print("Test Loss: ", score[0]) 
print("Test Accuracy: ", score[1])



Epoch 1/100
313/313 [==============================] - 5s 11ms/step - loss: 2.4367 - accuracy: 0.1678 - val_loss: 2.0618 - val_accuracy: 0.2339
Epoch 2/100
313/313 [==============================] - 3s 9ms/step - loss: 2.0360 - accuracy: 0.2843 - val_loss: 1.9326 - val_accuracy: 0.2778
Epoch 3/100
313/313 [==============================] - 3s 9ms/step - loss: 1.9131 - accuracy: 0.3207 - val_loss: 1.9252 - val_accuracy: 0.2842
Epoch 4/100
313/313 [==============================] - 3s 9ms/step - loss: 1.8836 - accuracy: 0.3397 - val_loss: 1.8559 - val_accuracy: 0.3192
Epoch 5/100
313/313 [==============================] - 3s 9ms/step - loss: 1.8312 - accuracy: 0.3542 - val_loss: 1.8296 - val_accuracy: 0.3421
Epoch 6/100
313/313 [==============================] - 3s 9ms/step - loss: 1.8093 - accuracy: 0.3625 - val_loss: 1.8331 - val_accuracy: 0.3145
Epoch 7/100
313/313 [==============================] - 3s 9ms/step - loss: 1.8033 - accuracy: 0.3741 - val_loss: 1.8259 - val_accuracy: 0.338

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
2cwt (Conv2D)                (None, 22, 22, 15)        5460      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 11, 11, 15)        0         
_________________________________________________________________
dropout (Dropout)            (None, 11, 11, 15)        0         
_________________________________________________________________
flatten (Flatten)            (None, 1815)              0         
_________________________________________________________________
tea_2 (Tea)                  (None, 120)               435720    
_________________________________________________________________
additive_pooling (AdditivePo (None, 10)                0     

In [10]:
orig_weights, constrained_weights = constrain_weights(model)
model.set_weights(constrained_weights)

print_weights(model)

print("Post-Ternary Constraint Accuracy: ")

# Evaluate the constained weight model
score = model.evaluate(x_test, y_test, verbose=0)

test_predictions = model.predict(x_test)

print("Test Loss: ", score[0])
print("Test Accuracy: ", score[1])

# Restore the original weights temporarily so we can evaluate them
model.set_weights(orig_weights)

print("Original Floating-Point Accuracy: ")

# Evaluate the original, floating-point weight model
float_score = model.evaluate(x_test, y_test, verbose=0)

print("Test Loss: ", float_score[0])
print("Test Accuracy: ", float_score[1])

print("Accuracy loss due to train-then-constrain: " ,float_score[1] - score[1])

2cwt/kernel:0 weights: 
(11, 11, 3, 15)
0.0-1.00.0 0.00.00.0 0.00.00.0 -1.0-1.00.0 -1.00.00.0 0.00.00.0 0.0-1.0-1.0 0.0-1.00.0 0.00.0-1.0 0.00.0-1.0 0.00.0-1.0 
0.00.00.0 1.0-1.00.0 -1.0-1.01.0 0.00.00.0 -1.0-1.00.0 0.00.00.0 0.00.0-1.0 -1.0-1.0-1.0 0.00.00.0 0.00.0-1.0 1.01.0-1.0 
0.00.00.0 -1.0-1.00.0 -1.0-1.00.0 -1.00.00.0 -1.00.00.0 -1.00.00.0 0.0-1.0-1.0 -1.0-1.0-1.0 0.0-1.0-1.0 1.00.0-1.0 0.01.00.0 
-1.0-1.00.0 -1.0-1.00.0 0.0-1.00.0 0.0-1.0-1.0 -1.0-1.00.0 0.00.00.0 0.0-1.0-1.0 -1.0-1.0-1.0 0.00.00.0 1.01.00.0 1.00.00.0 
-1.0-1.0-1.0 -1.00.01.0 -1.0-1.00.0 1.00.0-1.0 0.00.00.0 -1.0-1.0-1.0 -1.0-1.0-1.0 0.0-1.00.0 1.00.00.0 1.00.01.0 1.00.00.0 
-1.0-1.00.0 -1.0-1.00.0 -1.0-1.00.0 -1.0-1.00.0 0.0-1.0-1.0 -1.0-1.0-1.0 -1.0-1.0-1.0 0.00.00.0 0.00.00.0 1.01.00.0 1.00.00.0 
-1.0-1.00.0 -1.0-1.0-1.0 -1.0-1.0-1.0 -1.0-1.0-1.0 -1.0-1.0-1.0 -1.00.0-1.0 1.00.00.0 0.00.01.0 0.01.01.0 1.00.00.0 1.00.00.0 
-1.00.01.0 0.00.00.0 0.0-1.00.0 -1.0-1.0-1.0 -1.0-1.0-1.0 0.00.0-1.0 1.00.0-1.0 0.00.00

In [11]:
inputsCWT = Input(shape=(32,32,3))

# networkCWT = CWTConv2D(filters=10,
# 				  kernel_size=(3,3),
# 				  strides=(1,1)
# 				  #activation='relu',
# 				  #kernel_regularizer=regularizers.l1(l=0.1),
# 				  #use_bias=True,
# 				  )(inputsCWT)

network2CWT = CWTConv2D(filters=15,
				  kernel_size=(11,11),
				  strides=(1,1)
				  #activation='relu',
				  #kernel_regularizer=regularizers.l1(l=0.1),
				  #use_bias=True,
				  )(inputsCWT)

pooledcwt = CWTMaxPooling2D(pool_size=(2,2),strides=(2,2))(network2CWT)

dropoutCWT = Dropout(0.4)(pooledcwt)

flattenedCWT = Flatten()(dropoutCWT)

tea = Tea(units=120, name="tea_22")(flattenedCWT)

poolingCWT = AdditivePooling(10)(tea)

predictionsCWT = Activation('softmax')(poolingCWT)

modelCWT = Model(inputs=inputsCWT, outputs=predictionsCWT)

modelCWT.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

x_train = x_train.reshape(-1, 32,32,3) 
x_test = x_test.reshape(-1, 32,32,3)

modelCWT.fit(x_train, y_train, batch_size=128, epochs=100, verbose=1, validation_split=0.2)

scoreCWT = modelCWT.evaluate(x_test, y_test, verbose=0)

print("Test Loss: ", scoreCWT[0]) 
print("Test Accuracy: ", scoreCWT[1])

Epoch 1/100
313/313 [==============================] - 4s 10ms/step - loss: 2.4810 - accuracy: 0.1320 - val_loss: 2.2008 - val_accuracy: 0.1595
Epoch 2/100
313/313 [==============================] - 3s 9ms/step - loss: 2.2022 - accuracy: 0.1891 - val_loss: 2.1316 - val_accuracy: 0.1923
Epoch 3/100
313/313 [==============================] - 3s 9ms/step - loss: 2.1403 - accuracy: 0.2253 - val_loss: 2.0811 - val_accuracy: 0.2364
Epoch 4/100
313/313 [==============================] - 3s 9ms/step - loss: 2.1066 - accuracy: 0.2433 - val_loss: 2.0530 - val_accuracy: 0.2524
Epoch 5/100
313/313 [==============================] - 3s 8ms/step - loss: 2.0648 - accuracy: 0.2622 - val_loss: 2.0433 - val_accuracy: 0.2543
Epoch 6/100
313/313 [==============================] - 3s 9ms/step - loss: 2.0330 - accuracy: 0.2766 - val_loss: 2.0146 - val_accuracy: 0.2772
Epoch 7/100
313/313 [==============================] - 3s 8ms/step - loss: 2.0235 - accuracy: 0.2877 - val_loss: 1.9962 - val_accuracy: 0.273

In [12]:
orig_weightsCWT, constrained_weightsCWT = constrain_weights_cwt(modelCWT)
modelCWT.set_weights(constrained_weightsCWT)
		

print("Post-Ternary Constraint Accuracy: ")

# Evaluate the constained weight model
scoreCWT = modelCWT.evaluate(x_test, y_test, verbose=0)

test_predictionsCWT = modelCWT.predict(x_test)

print("Test Loss: ", scoreCWT[0])
print("Test Accuracy: ", scoreCWT[1])

# Restore the original weights temporarily so we can evaluate them
modelCWT.set_weights(orig_weightsCWT)

print("Original Floating-Point Accuracy: ")

# Evaluate the original, floating-point weight model
float_scoreCWT = modelCWT.evaluate(x_test, y_test, verbose=0)

print("Test Loss: ", float_scoreCWT[0])
print("Test Accuracy: ", float_scoreCWT[1])

print("Accuracy loss due to train-then-constrain: " ,float_scoreCWT[1] - scoreCWT[1])

Post-Ternary Constraint Accuracy: 
Test Loss:  1.6675361394882202
Test Accuracy:  0.4041000008583069
Original Floating-Point Accuracy: 
Test Loss:  1.6675361394882202
Test Accuracy:  0.4041000008583069
Accuracy loss due to train-then-constrain:  0.0
